In [4]:

from __future__ import division
from scipy import stats
import numpy as np
from math import log
from compiler.ast import flatten


# This method computes entropy for information gain
def entropy(class_y):
    # Input:            
    # class_y         : list of class labels (0's and 1's)

    # TODO: Compute the entropy for a list of classes
    #
    # Example:
    #    entropy([0,0,0,1,1,1,1,1,1]) = 0.92
    log2 = lambda x: log(x) / log(2)
    labels = np.unique(class_y)
    entropy = 0

    y = np.asarray(class_y)
    for label in labels:
        count = len(y[y == label])
        p = count / len(class_y)
        entropy += -p * log2(p)

    return entropy



def try_partition_classes(X, y, split_attribute, split_val):
    y_left = []
    y_right = []

    X_arr = np.asarray(X, dtype=object)
    y_arr = np.array(y)

    # If split_attribute is numeric, then split on <= condition
    if not type(split_val)==str:
        idx_left = np.where(X_arr[:,split_attribute] <= split_val)[0]
        idx_right = np.where(X_arr[:,split_attribute] > split_val)[0]
        y_left = y_arr[idx_left].tolist()
        y_right = y_arr[idx_right].tolist()

    # If split_attribute is categorical, then split on split_val
    else:
        idx_left = np.where(X_arr[:,split_attribute] == split_val)[0]
        idx_right = np.where(X_arr[:,split_attribute] != split_val)[0]    
        y_left = y_arr[idx_left].tolist()
        y_right = y_arr[idx_right].tolist()

    return (y_left, y_right)




def partition_classes(X, y, split_attribute, split_val):
    # Inputs:
    #   X               : data containing all attributes
    #   y               : labels
    #   split_attribute : column index of the attribute to split on
    #   split_val       : either a numerical or categorical value to divide the split_attribute
    
    # TODO: Partition the data(X) and labels(y) based on the split value - BINARY SPLIT.
    # 
    # You will have to first check if the split attribute is numerical or categorical    
    # If the split attribute is numeric, split_val should be a numerical value
    # For example, your split_val could be the mean of the values of split_attribute
    # If the split attribute is categorical, split_val should be one of the categories.   
    #
    # You can perform the partition in the following way
    # Numeric Split Attribute:
    #   Split the data X into two lists(X_left and X_right) where the first list has all
    #   the rows where the split attribute is less than or equal to the split value, and the 
    #   second list has all the rows where the split attribute is greater than the split 
    #   value. Also create two lists(y_left and y_right) with the corresponding y labels.
    #
    # Categorical Split Attribute:
    #   Split the data X into two lists(X_left and X_right) where the first list has all 
    #   the rows where the split attribute is equal to the split value, and the second list
    #   has all the rows where the split attribute is not equal to the split value.
    #   Also create two lists(y_left and y_right) with the corresponding y labels.

    '''
    Example:
    
    X = [[3, 'aa', 10],                 y = [1,
         [1, 'bb', 22],                      1,
         [2, 'cc', 28],                      0,
         [5, 'bb', 32],                      0,
         [4, 'cc', 32]]                      1]
    
    Here, columns 0 and 2 represent numeric attributes, while column 1 is a categorical attribute.
    
    Consider the case where we call the function with split_attribute = 0 and split_val = 3 (mean of column 0)
    Then we divide X into two lists - X_left, where column 0 is <= 3  and X_right, where column 0 is > 3.
    
    X_left = [[3, 'aa', 10],                 y_left = [1,
              [1, 'bb', 22],                           1,
              [2, 'cc', 28]]                           0]
              
    X_right = [[5, 'bb', 32],                y_right = [0,
               [4, 'cc', 32]]                           1]

    Consider another case where we call the function with split_attribute = 1 and split_val = 'bb'
    Then we divide X into two lists, one where column 1 is 'bb', and the other where it is not 'bb'.
        
    X_left = [[1, 'bb', 22],                 y_left = [1,
              [5, 'bb', 32]]                           0]
              
    X_right = [[3, 'aa', 10],                y_right = [1,
               [2, 'cc', 28],                           0,
               [4, 'cc', 32]]                           1]
               
    '''

    X_left = []
    X_right = []

    y_left = []
    y_right = []

    X_arr = np.asarray(X, dtype=object)
    y_arr = np.array(y)

    # If split_attribute is numeric, then split on <= condition
    if not type(split_val)==str:
        idx_left = np.where(X_arr[:,split_attribute] <= split_val)[0]
        idx_right = np.where(X_arr[:,split_attribute] > split_val)[0]
        X_left = X_arr[idx_left].tolist()
        X_right = X_arr[idx_right].tolist()
        y_left = y_arr[idx_left].tolist()
        y_right = y_arr[idx_right].tolist()

    # If split_attribute is categorical, then split on split_val
    else:
        idx_left = np.where(X_arr[:,split_attribute] == split_val)[0]
        idx_right = np.where(X_arr[:,split_attribute] != split_val)[0]   
        X_left = X_arr[idx_left].tolist()
        X_right = X_arr[idx_right].tolist() 
        y_left = y_arr[idx_left].tolist()
        y_right = y_arr[idx_right].tolist()

    return (X_left, X_right, y_left, y_right)


def information_gain(previous_y, current_y):
    # Inputs:
    #   previous_y: the distribution of original labels (0's and 1's)
    #   current_y:  the distribution of labels after splitting based on a particular
    #               split attribute and split value

    # TODO: Compute and return the information gain from partitioning the previous_y labels
    # into the current_y labels.
    # You will need to use the entropy function above to compute information gain
    # Reference: http://www.cs.cmu.edu/afs/cs.cmu.edu/academic/class/15381-s06/www/DTs.pdf

    """
    Example:

    previous_y = [0,0,0,1,1,1]
    current_y = [[0,0], [1,1,1,0]]

    info_gain = 0.45915
    """

    info_gain = 0
    y_left = current_y[0]
    y_right = current_y[1]
    info_gain = entropy(previous_y) - (entropy(y_left) * (len(y_left) / len(flatten(current_y))) + (entropy(y_right) * (len(y_right) / len(flatten(current_y)))))

    return info_gain


# WORK IN PROGRESS BELOW:

# def get_split(X,y):
#
#     for index in range(len(X)):
#         for row in X:
#             groups_xleft, groups_xright, groups_yleft, groups_yright = partition_classes(X,y,index,row[index])
#             ig = information_gain(y, [groups_yleft, groups_yright])


/Users/david/anaconda3/envs/py27/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: The compiler package is deprecated and removed in Python 3.x.
  


In [5]:
'''
Majority of the code borrowed from previous work done for CSE 6242 (Data Visualization)
Help from:
Overall Structure: http://www.onlamp.com/pub/a/python/2006/02/09/ai_decision_trees.html?page=3
https://github.com/eriklindernoren/ML-From-Scratch/blob/master/mlfromscratch/supervised_learning/decision_tree.py
https://gist.github.com/iamaziz/02491e36490eb05a30f8

https://codereview.stackexchange.com/questions/109089/id3-decision-tree-in-python
'''


class DecisionTree(object):
    def __init__(self):
        # Initializing the tree as an empty dictionary or list, as preferred
        # self.tree = []
        self.tree = {}
        self.threshold = 0.1

    def learn(self, X, y):
        # TODO: Train the decision tree (self.tree) using the the sample X and labels y
        # You will have to make use of the functions in utils.py to train the tree

        # One possible way of implementing the tree:
        #    Each node in self.tree could be in the form of a dictionary:
        #    https://docs.python.org/2/library/stdtypes.html#mapping-types-dict
        #    For example, a non-leaf node with two children can have a 'left' key and  a
        #    'right' key. You can add more keys which might help in classification
        #    (eg. split attribute and split value)
        gain, index, value = self.try_split(X, y)
        self.tree = self.get_split(X, y, index, value)
        # print(self.tree)
        self.split(self.tree, 1)

    # Select the best split point for a dataset
    '''
    Changelog:
    Splitting function is now (ironically) split into two functions (try_split & get_split).  

        try_split: 
        Required efficiency gains by creating X_arr (numpy is much more efficient).
        Another efficiency gain is minimizing looping.  Previously, we looped every column in X and then every
        row in X.  Now, we only loop *once* every *unique* value in X.  


    '''

    def try_split(self, X, y):
        b_index, b_value, b_gain = -1, -1, float('-inf')

        X_arr = np.asarray(X, dtype=object)
        number_of_attr = X_arr.shape[1]
        idx = np.random.choice(range(number_of_attr), int(np.floor(0.4 * number_of_attr)), replace=False)
        for index in idx:
            partition_values = np.unique(X_arr[:, index], return_counts=False)
            if not type(partition_values[0]) == str:
                partition_values = np.percentile(partition_values, [10 * i for i in range(1, 10, 2)])

            gain = np.array(
                [information_gain(y, try_partition_classes(X, y, index, value)) for value in partition_values])
            max_gain = max(gain)
            if max_gain < self.threshold:
                continue
            else:
                if max_gain > b_gain:
                    b_index, b_value, b_gain = index, partition_values[np.argmax(gain)], max_gain

        return (b_gain, b_index, b_value)

    def get_split(self, X, y, index, value):
        groups_xleft, groups_xright, groups_yleft, groups_yright = partition_classes(X, y, index, value)
        return {'index': index, 'value': value, 'groups_xleft': groups_xleft, 'groups_xright': groups_xright, \
                'groups_yleft': groups_yleft, 'groups_yright': groups_yright}

    # Create a terminal node value
    def to_terminal(self, y):
        return max(set(y), key=y.count)

    def check_identical(self, X):
        X_array = np.asarray(X, dtype=object)
        for m in range(X_array.shape[1]):
            if len(set(X_array[:, m])) > 1:
                return False
        return True

    # Create child splits for a node or make terminal
    def split(self, node, depth, max_depth=5, min_size=5):
        left_X, right_X, left_y, right_y = node['groups_xleft'], node['groups_xright'], node['groups_yleft'], node[
            'groups_yright']
        # del(node['groups_xleft'], node['groups_xright'], node['groups_yleft'],  node['groups_yright'])
        # check for a no split
        if (len(left_y) == 0 or len(right_y) == 0):
            node['left'] = node['right'] = self.to_terminal(left_y + right_y)
            return
        if (len(set(left_y + right_y)) == 1):
            node['left'] = node['right'] = self.to_terminal(left_y + right_y)
            return
        if (self.check_identical(left_X + right_X)):
            node['left'] = node['right'] = self.to_terminal(left_y + right_y)
            return
        # check for max depth
        if depth >= max_depth:
            node['left'], node['right'] = self.to_terminal(left_y), self.to_terminal(right_y)
            return
        # process left child
        if len(left_X) <= min_size:
            node['left'] = self.to_terminal(left_y)
        else:
            gain, index, value = self.try_split(left_X, left_y)
            if gain > self.threshold:
                node['left'] = self.get_split(left_X, left_y, index, value)
                self.split(node['left'], depth + 1, max_depth, min_size)
            else:
                node['left'] = self.to_terminal(left_y)
        # process right child
        if len(right_X) <= min_size:
            node['right'] = self.to_terminal(right_y)
        else:
            gain, index, value = self.try_split(right_X, right_y)
            if gain > self.threshold:
                node['right'] = self.get_split(right_X, right_y, index, value)
                self.split(node['right'], depth + 1, max_depth, min_size)
            else:
                node['right'] = self.to_terminal(right_y)

    def classify(self, record):
        # TODO: classify the record using self.tree and return the predicted label
        node = self.tree
        return self.predict(node, record)

    def predict(self, node, record):
        if isinstance(record[node['index']], str):
            if record[node['index']] == node['value']:
                if isinstance(node['left'], dict):
                    return self.predict(node['left'], record)
                else:
                    return node['left']
            else:
                if isinstance(node['right'], dict):
                    return self.predict(node['right'], record)
                else:
                    return node['right']
        else:
            if record[node['index']] < node['value']:
                if isinstance(node['left'], dict):
                    return self.predict(node['left'], record)
                else:
                    return node['left']
            else:
                if isinstance(node['right'], dict):
                    return self.predict(node['right'], record)
                else:
                    return node['right']




In [29]:
import csv
import ast
class RandomForest(object):

    def __init__(self, num_trees):
        # Initialization done here
        self.num_trees = num_trees
        self.decision_trees = [DecisionTree() for i in range(num_trees)]
        
    def _bootstrapping(self, XX, n):
        # Reference: https://en.wikipedia.org/wiki/Bootstrapping_(statistics)
        #
        # TODO: Create a sample dataset of size n by sampling with replacement
        #       from the original dataset XX.
        # Note that you would also need to record the corresponding class labels
        # for the sampled records for training purposes. 

        idx = np.random.choice(range(len(XX)), n, replace=True)
        samples = [XX[id][:-1] for id in idx] # sampled dataset
        labels = [XX[id][-1] for id in idx]  # class labels for the sampled records
        return (samples, labels)


    def bootstrapping(self, XX):
        # Initializing the bootstap datasets for each tree
        for i in range(self.num_trees):
            data_sample, data_label = self._bootstrapping(XX, len(XX))
            self.bootstraps_datasets.append(data_sample)
            self.bootstraps_labels.append(data_label)


    def fitting(self):
        # TODO: Train `num_trees` decision trees using the bootstraps datasets
        # and labels by calling the learn function from your DecisionTree class.
        for i in range(self.num_trees):            
            print "Build the %dth tree" % (i+1)
            X = self.bootstraps_datasets[i]
            y = self.bootstraps_labels[i]
            tree =  self.decision_trees[i]
            tree.learn(X, y)      


    def voting(self, X):
        y = []

        for record in X:
            # Following steps have been performed here:
            #   1. Find the set of trees that consider the record as an 
            #      out-of-bag sample.
            #   2. Predict the label using each of the above found trees.
            #   3. Use majority vote to find the final label for this recod.
            votes = []
            for i in range(len(self.bootstraps_datasets)):
                dataset = self.bootstraps_datasets[i]
                if record not in dataset:
                    OOB_tree = self.decision_trees[i]
                    effective_vote = OOB_tree.classify(record)
                    votes.append(effective_vote)


            counts = np.bincount(votes)
            
            if len(counts) == 0:
                # TODO: Special case 
                #  Handle the case where the record is not an out-of-bag sample
                #  for any of the trees. 
                votes = [tree.classify(record) for tree in self.decision_trees]
                counts = np.bincount(votes)
                y = np.append(y, np.argmax(counts))
            else:
                y = np.append(y, np.argmax(counts))

        return y

# DO NOT change the main function apart from the forest_size parameter!
def main():
    X = list()
    y = list()
    XX = list()  # Contains data features and data labels
    numerical_cols = set([1, 2, 7, 10, 13, 14, 15]) # indices of numeric attributes (columns)

    # Loading data set
    print 'reading data'
    with open("data.csv") as f:
        next(f, None)

        for line in csv.reader(f, delimiter=","):
            xline = []
            for i in range(len(line)):
                if i in numerical_cols:
                    xline.append(ast.literal_eval(line[i]))
                else:
                    xline.append(line[i])

            X.append(xline[:-1])
            y.append(xline[-1])
            XX.append(xline[:])

    # TODO: Initialize according to your implementation
    # VERY IMPORTANT: Minimum forest_size should be 10
    forest_size = 50
    
    # Initializing a random forest.
    randomForest = RandomForest(forest_size)

    # Creating the bootstrapping datasets
    print 'creating the bootstrap datasets'
    randomForest.bootstrapping(XX)

    # Building trees in the forest
    print 'fitting the forest'
    randomForest.fitting()

    # Calculating an unbiased error estimation of the random forest
    # based on out-of-bag (OOB) error estimate.
    y_predicted = randomForest.voting(X)

    # Comparing predicted and true labels
    results = [prediction == truth for prediction, truth in zip(y_predicted, y)]

    # Accuracy
    accuracy = float(results.count(True)) / float(len(results))

    print "accuracy: %.4f" % accuracy
    print "OOB estimate: %.4f" % (1-accuracy)


In [37]:
a = RandomForest(10)._bootstrapping

TypeError: _bootstrapping() takes exactly 3 arguments (1 given)

In [36]:
a

<bound method RandomForest._bootstrapping of <__main__.RandomForest object at 0x10eccf250>>

TypeError: main() takes no arguments (1 given)